# NOPT042 Constraint programming: Tutorial 1 - Introduction to Picat

## Installation

You can install [Picat](http://picat-lang.org/) like this (check if there's a newer version of Picat):

```bash
cd ~
wget http://picat-lang.org/download/picat328_linux64.tar.gz
tar -xf picat328_linux64.tar.gz
```

Then add the executable to `$PATH` (assuming we use bash):

```bash
echo 'export PATH="$HOME/Picat:$PATH"' >> ~/.bashrc
source ~/.bashrc
```

Then the command `picat` runs the Picat interpreter.

If you want to execute the notebooks, install [Jupyter Notebook](https://jupyter.org/) with [ipicat extension](https://pypi.org/project/picat-kernel/) (if you want to install them locally, add `--user`):

```bash
pip install jupyter
pip install ipicat
```

Then run `jupyter notebook`. Once the extension is loaded you can use `%%picat` cell magic or execute picat files: `%picat -e hello-world.pi`.

In [2]:
%load_ext ipicat

<IPython.core.display.Javascript object>

Picat version 3.2#8


## Introductory examples

### Hello world

In [3]:
%%picat
main =>
    println("Hello, World!").

Hello, World!



In [4]:
%picat -e hello-world.pi
# alternatively:
!picat hello-world.pi

Hello, World!

Hello, World!


### Command-line arguments

In [5]:
# This doesn't work at the moment
# %picat -e hello-world.pi Alice
!picat hello-world.pi Alice
!picat hello-world.pi Alice Bob Carol Dave

Hello, Alice! You are my favourite student.
Hello, Alice and Bob and Carol and Dave! You are my favourite students.


In [6]:
%%bash
cat hello-world.pi

import util.

main =>
    println("Hello, World!").

main([Name]) =>
    printf("Hello, %s! You are my favourite student.\n", Name).

main(ARGS) =>
    Names = ARGS.join(" and "),
    printf("Hello, %s! You are my favourite students.\n", Names).

### Example: Fibonacci sequence

### Example: Quicksort

In Jupyter, use `%%picat -e predicate_name` to define a predicate from a cell.

In [30]:
%%picat -n qsort
qsort([]) = [].
qsort([H | T]) = qsort([E : E in T, E =< H]) ++ [H] ++  qsort([E : E in T, E > H]).

Alternative version:

In [32]:
%%picat -n qsort
qsort(L) = Lsorted => 
    if L = [] then
        Lsorted = []
    else
        L = [H | T],
        Lsorted = qsort([E : E in T, E =< H]) ++ [H] ++  qsort([E : E in T, E > H]).

In [33]:
%%picat
main => L = qsort([5, 2, 6, 4, 1, 3]), println(L).

[1,2,3,4,5,6]



In [24]:
!picat qsort/qsort.pi

List [5,2,6,4,1,3] after sorting is [1,2,3,4,5,6].


In [25]:
!picat qsort/qsort.pi [5,2,6,4,1,3]

[1,2,3,4,5,6]


### Reading and writing files

In [26]:
!cat qsort/assorted.lists

[2, 1]
[5, 2, 6, 4, 1, 3]
[44, 11, 29, 53, 59, 70, 63, 68, 16, 30, 95, 9, 55, 71, 84, 81, 64, 46, 26, 89, 15, 40, 22, 97, 39]

In [28]:
!picat qsort/qsort.pi qsort/assorted.lists qsort/sorted.lists
!cat qsort/sorted.lists

[1,2]
[1,2,3,4,5,6]
[9,11,15,16,22,26,29,30,39,40,44,46,53,55,59,63,64,68,70,71,81,84,89,95,97]


In [29]:
!cat qsort/qsort.pi

qsort([])    = [].
qsort([H|T]) = qsort([E : E in T, E =< H]) ++ [H] ++ qsort([E : E in T, E > H]).

main =>
    L = [5, 2, 6, 4, 1, 3],    
    printf("List %w after sorting is %w.\n", L, qsort(L)).

main([Lstring]) =>
    L = parse_term(Lstring),
    println(qsort(L)).

main([InputPath, OutputPath]) =>
    Lines = read_file_lines(InputPath),
    OutputFile = open(OutputPath, write),
    foreach(I in 1..Lines.length)
        L = parse_term(Lines[I]),
        writeln(OutputFile, qsort(L))
    end.


### TPK algorithm

The TPK algorithm is an artificial problem designed by Trabb Pardo & Knuth to showcase the syntax of a given programming language (see [Wikipedia](https://en.wikipedia.org/wiki/TPK_algorithm)):
```markdown
ask for 11 numbers to be read into a sequence S
reverse sequence S
for each item in sequence S
    call a function to do an operation
    if result overflows
        alert user
    else
        print result
```
The following Picat implementation is from [here](https://www.linuxjournal.com/content/introduction-tabled-logic-programming-picat).

In [1]:
!picat tpk/tpk.pi < -5 1.0 -2.345 42.0001 1e26 5.24e-17 0.1 0.01 0.11001 2 3 

/bin/bash: -5: No such file or directory


## An overview of Picat

Examples in this section are mostly adapted from or inspired by the [Picat Book][1], [Picat Guide][2], [AAA2017 tutorial][3], and [examples][4]. More resources are available [here][5].

[1]: http://picat-lang.org/picatbook2015.html
[2]: https://web.mit.edu/picat_v1.9/picat_guide.pdf
[3]: http://ktiml.mff.cuni.cz/~bartak/AAAI2017/
[4]: http://picat-lang.org/exs/exs.pi
[5]: http://picat-lang.org/resources.html

TODO, see the [slides](http://ktiml.mff.cuni.cz/~bartak/AAAI2017/slides.pdf).

## Playground

In [38]:
%%picat
fib(N, F) => 
    if (N = 0) then 
        F = 0
    elseif (N = 1) then
        F = 1
    else
        fib(N - 1, F1),
        fib(N - 2, F2),
        F = F1 + F2
    end.
    
main =>
    fib(42, F),
    println(F).

267914296



In [37]:
fib(N, F) => 
    if (N = 0) then 
        F = 0
    elseif (N = 1) then
        F = 1
    else
        fib(N - 1, F1),
        fib(N - 2, F2),
        F = F1 + F2
    end.
    
main =>
    fib(11, F),
    println(F).

SyntaxError: cannot assign to function call (971927783.py, line 1)

In [8]:
%%time
%%picat

% % try uncommenting the following line
% table 

fibp(0,F) => F = 0.
fibp(1,F) => F = 1.
fibp(N,F),N>1 => fibp(N-1,F1), fibp(N-2,F2), F = F1+F2.

main => fibp(42,F), println(F).

267914296

CPU times: user 5.34 ms, sys: 124 µs, total: 5.46 ms
Wall time: 14.3 ms
